# Solving Sudoku Puzzle with Pyomo and Excel

This formulation is based on the chapter 5.4.2 from Pyomo documentation.

$$
\begin{align}
    \text{s.t.} & \sum_{r \in ROWS} y_{r,c,v} = 1 \quad & \forall c \in COLS, v \in VALUES \\
                & \sum_{c \in COLS} y_{r,c,v} = 1 \quad & \forall r \in ROWS, v \in VALUES \\
                & \sum_{v \in VALUES} y_{r,c,v} = 1 \quad & \forall r \in ROWS, c \in COLS \\
                & \sum_{r=1}^{3} \sum_{c=1}^{3} y_{(r + U),(c + W),v} & U, W \in \{0,3,6\} \\
                & y_{r,c,v} \in \{0,1\} \quad & \forall r \in ROWS, c \in COLS, v \in VALUES
\end{align}
$$

In [1]:
import pandas as pd
import pyomo.environ as pyo

In [2]:
board = pd.read_excel('./sudoku_input_0.xlsx', header=None)

In [3]:
def sudoku_model(board):

    model = pyo.ConcreteModel()

    # store the starting board
    # board = pd.read_excel('./sudoku_input_0.xlsx', header=None)
    board.index += 1
    board.columns += 1
    display(board)

    # create sets for rows, columns and subsquares
    model.ROWS = pyo.Set(initialize=board.index)
    model.COLS = pyo.Set(initialize=board.columns)
    model.VALUES = pyo.Set(initialize=pyo.RangeSet(1, 9))
    
    model.U = pyo.Set(initialize=[0, 3, 6])
    model.W = pyo.Set(initialize=[0, 3, 6])

    # fix variables based on the current board
    for r in model.ROWS:
        for c in model.COLS:
            if board.loc[r, c] in model.VALUES:
                model.y[r, c, board.loc[r, c]].fix(1)

    # exactly one number in each row
    def row_cstr(model, c, v):
        return sum(model.y[:, c, v]) == 1
    
    model.row_cstr = pyo.Constraint(model.COLS, model.VALUES, rule=row_cstr)

    # exactly one number in each column
    def col_cstr(model, r, v):
        return sum(model.y[r, :, v]) == 1

    model.col_cstr = pyo.Constraint(model.ROWS, model.VALUES, rule=col_cstr)

    # exactly one number in each subsquare
    def subsquares_cstr(model, u, w, v):
        return sum(model.y[r+u, c+w, v] for r in range(1, 4) for c in range(1, 4)) == 1

    model.subsquares_cstr = pyo.Constraint(model.U, model.W, model.VALUES, rule=subsquares_cstr)

    # exactly one number in each cell
    def value_cstr(model, r, c):
        return sum(model.y[r, c, :]) == 1

    model.value_cstr = pyo.Constraint(model.ROWS, model.COLS, rule=value_cstr)

    # objective function - feasibility problem so we just make it a constant
    model.obj = pyo.Objective(expr=1.0)

## Integer cut

The integer cut uses two sets. The first set $S_0$ consists of indices for those variables whose current solution is 0, and the second set $S_1$ consists of indices for those variables whose current solution is 1. Given the two sets, the integer cut constraint would prevent such a solution from appeating twice.

$$
\begin{align}
    \sum_{(r,c,v) \in S_0} y_{r,c,v} + \sum_{(r,c,v) \in S_1} (1 - y_{r,c,v}) \ge 1
\end{align}
$$

In [13]:
# adding a new integer cur to the model

def add_integer_cut(model):
    # add the ConstraintList to store the IntegerCuts if it does not already exist
    if not hasattr(model, 'IntegerCuts'):
        model.IntegerCuts = pyo.ConstraintList()

    # add the integer cut corresponding to the current solution in the model
    cut_expr = 0.0
    for r in model.ROWS:
        for c in model.COLS:
            for v in model.VALUES:
                if not model.y[r, c, v].fixed:
                    if pyo.value(model.y[r, c, v]) >= 0.5:
                        cut_expr += (1.0 - model.y[r, c, v])
                    else:
                        cut_expr += model.y[r, c, v]
    model.IntegerCuts.add(cut_expr >= 1)


In [14]:
# prints the solution stored in the model

def print_solution(model):
    for r in model.ROWS:
        print(' '.join(str(v) for c in model.COLS
                       for v in model.VALUES
                       if pyo.value(model.y[r, c, v]) >= 0.5))

In [22]:
solution_count = 0
while 1:
    with pyo.SolverFactory("glpk") as opt:
        solution = opt.solve(model)
        if solution.solver.termination_condition != pyo.TerminationCondition.optimal:
            print("All board solutions have been found")
            break

    solution_count += 1

    add_integer_cut(model)

    print("Solution #%d" % (solution_count))
    print_solution(model)

print('Number of solutions found = %d' % (max(solution_count)))

    solver failure.


Number of solutions found = 1
Solution #1
2 9 8 4 6 5 1 7 3
1 6 5 7 9 3 8 2 4
3 4 7 1 2 8 9 6 5
7 8 2 5 3 1 4 9 6
6 1 9 2 8 4 5 3 7
5 3 4 9 7 6 2 1 8
8 7 1 3 4 2 6 5 9
4 5 3 6 1 9 7 8 2
9 2 6 8 5 7 3 4 1
    solver failure.
Number of solutions found = 2
Solution #2
9 4 6 7 3 2 1 5 8
8 1 3 4 5 6 2 9 7
7 5 2 1 8 9 3 6 4
4 3 8 6 1 7 5 2 9
5 2 7 3 9 4 8 1 6
1 6 9 8 2 5 4 7 3
6 8 1 2 7 3 9 4 5
3 7 5 9 4 1 6 8 2
2 9 4 5 6 8 7 3 1
    solver failure.
Number of solutions found = 3
Solution #3
3 6 7 4 2 9 8 5 1
2 8 5 1 6 7 4 9 3
4 9 1 8 3 5 6 7 2
5 1 8 7 9 4 2 3 6
9 2 3 5 1 6 7 4 8
7 4 6 3 8 2 9 1 5
6 5 9 2 4 3 1 8 7
1 3 2 9 7 8 5 6 4
8 7 4 6 5 1 3 2 9
    solver failure.
Number of solutions found = 4
Solution #4
5 6 2 4 8 9 7 3 1
1 8 4 5 7 3 9 2 6
7 9 3 1 6 2 5 8 4
8 1 7 6 2 4 3 5 9
2 5 6 3 9 1 8 4 7
4 3 9 8 5 7 1 6 2
3 7 1 2 4 8 6 9 5
6 2 8 9 1 5 4 7 3
9 4 5 7 3 6 2 1 8
    solver failure.
Number of solutions found = 5
Solution #5
6 1 9 8 2 5 3 7 4
7 4 8 6 1 3 9 5 2
2 5 3 9 4 7 8 1 6
5 6 2 1 

KeyboardInterrupt: 

In [ ]:
# prints the solution stored in the model

def print_solution(model):
    for r in model.ROWS:
        print(' '.join(str(v) for c in model.COLS
                       for v in model.VALUES
                       if pyo.value(model.y[r, c, v]) >= 0.5))

In [26]:
def solution_to_datagrame(model):
    grid = []

    for r in model.ROWS:
        row_values = [] * 9 # initialize an empty row
        row_values.append(str(v) for c in model.COLS
                          for v in model.VALUES
                          if pyo.value(model.y[r, c, v]) >= 0.5)
        



NameError: name 'grid' is not defined

In [16]:
def solution_to_dataframe(model):
    grid = []

    for r in model.ROWS:
        row_values = [] * 9 # initialize an ampty row
        for c in model.COLS:
            for v in model.VALUES:
                if pyo.value(model.y[r, c, v]) >= 0.5:
                    row_values.append(v)
        grid.append(row_values)
    
    df = pd.DataFrame(grid)
    return df

solution_df = solution_to_dataframe(model)
print(solution_df)

   0  1  2  3  4  5  6  7  8
0  1  3  6  7  4  5  8  2  9
1  4  9  8  2  1  6  5  3  7
2  5  7  2  9  8  3  4  6  1
3  7  1  5  4  9  2  3  8  6
4  8  2  3  6  5  1  7  9  4
5  9  6  4  8  3  7  2  1  5
6  2  4  1  3  7  9  6  5  8
7  6  8  9  5  2  4  1  7  3
8  3  5  7  1  6  8  9  4  2


In [21]:
wb = Workbook()

for idx, solution in enumerate(solutions, start=1):
    solution_df = solution_to_dataframe(solution)
    sheet_name = f"Solution_{idx}"
    ws = wb.create_sheet(title=sheet_name)
    for row in dataframe_to_rows(solution_df, index=False, header=False):
        ws.append(row)

# Remove the default sheet created and save the workbook
del wb['Sheet']
wb.save('sudoku_solutions.xlsx')

NameError: name 'solutions' is not defined

In [18]:
with pd.ExcelWriter('./sudoku_output.xlsx') as writer:
    solution_df.to_excel(writer, sheet_name='Solution #%d' % (solution_count))

In [ ]:
output_dict = {}

df = pd.DataFrame()

for r in model.ROWS:
        print(' '.join(str(v) for c in model.COLS
                       for v in model.VALUES
                       if pyo.value(model.y[r, c, v]) >= 0.5))


for i in solution_count:
    lines = solution.strip()
    
for r in model.ROWS:
    for c in model.COLS:
        for v in model.VALUES:
            if pyo.value(model.y[r, c, v]) >= 0.5:
                output_dict[(r, c)] = v